In [1]:
import torch
import numpy as np
import dgl
import math
from extension.data_process import distance, point_to_line_distance
from extension.data_process import train_indoor_node_location, train_outdoor_node_location, bin_indoor_node_location, bin_outdoor_node_location, ran_indoor_node_location, ran_outdoor_node_location

Using backend: pytorch


In [2]:
import cv2

cap = cv2.VideoCapture('D:/GCN/graphnet/data/室外定位1/30-1/output.mp4')
frame = 0
cap.set(1,1)
ret,frame=cap.read()

for point in bin_outdoor_node_location[0]:
    cv2.line(frame, (int(bin_outdoor_node_location[0][0][0]), -int(bin_outdoor_node_location[0][0][1])), (int(point[0]), -int(point[1])), (255, 0, 0), 2)
    

for point in ran_outdoor_node_location[0]:
    cv2.line(frame, (int(ran_outdoor_node_location[0][0][0]), -int(ran_outdoor_node_location[0][0][1])), (int(point[0]), -int(point[1])), (0, 255, 0), 2)
    
while(True):
    cv2.imshow("video", frame)
    c = cv2.waitKey(1)
    if c == 27:
        # cv2.imwrite('.\\indoor2.png', frame)
        break
cv2.destroyAllWindows()
print()

## 制作训练数据

In [ ]:
line_number = 4
op_type = ['train', 'bin_test', 'ran_test']
# 根据操作类型确定原始数据的来源以及生成文件的保存路径
for op in op_type:
    load_str = 'D:\\GCN\\graphnet\\data\\train_data\\' if op == 'train' else 'D:\\GCN\\graphnet\\data\\test_data2\\'
    graph_out_folder = ('.\\train_data\\' if op == 'train' else '.\\bin_test_data\\' if op == 'bin_test' else '.\\ran_test_data\\')
    max_sc_no = 6 if op == 'train' else 4

    for scene in ['indoor', 'outdoor']:
        for numid in range(max_sc_no):
            # 原始数据的路径
            time_path = load_str + scene + '\\30-' + str(numid + 1) + '\\dataset\\time.npy'
            feature_path = load_str + scene + '\\30-' + str(numid + 1) + '\\dataset\\feature.npy'
            obstacle_path = load_str + scene + '\\30-' + str(numid + 1) + '\\dataset\\obstacle.npy'
            label_path = load_str + scene + '\\30-' + str(numid + 1) + '\\dataset\\label.npy'
            
            # 生成数据的保存路径
            graph_path = graph_out_folder + scene + '_30-' + str(numid + 1) + '_graphs.dgl'
            if op == 'train' and scene == 'indoor':
                node_location = train_indoor_node_location[numid]
            if op == 'train' and scene == 'outdoor':
                node_location = train_indoor_node_location[numid]
            if op == 'bin_test' and scene == 'indoor':
                node_location = bin_indoor_node_location[numid]
            if op == 'bin_test' and scene == 'outdoor':
                node_location = bin_indoor_node_location[numid]
            if op == 'ran_test' and scene == 'indoor':
                node_location = ran_indoor_node_location[numid]
            if op == 'ran_test' and scene == 'outdoor':
                node_location = ran_indoor_node_location[numid]

            ti = np.load(time_path, allow_pickle=True)
            fe = np.load(feature_path, allow_pickle=True)

            # fe.shape:(n, 4, 3)其中 n 表示有 n 个时刻的数据，4行3列中每一行表示对应链路，第一列表示对应链路真实的rssi值，第二列表示与基准做差之后的结果，第三列表示etx
            for j in range(4):
                for i in range(len(fe)):
                    
                    # rssi, -1, -1
                    fe[i][j][2] = -1 # 将fe第三列的数据全部置为-1作为后续备用
                    if -100 < fe[i][j][0] < -70: # 对于-100到-70之间的数据，将其rssi + 10, 相应的与基准的差要减10
                        fe[i][j][0] += 10
                        fe[i][j][1] -= 10
                    while fe[i][j][0] == -100:
                        fe[i][j][0] = fe[i - 1][j][0]
                        fe[i][j][1] = fe[i - 1][j][1]
                        i += 1
                    fe[i][j][1] = -1
            # doc = open('D:\\GCN\\debugTools\\npy_readed\\fe_after.txt', 'w', encoding='utf-8')  #打开一个存储文件，并依次写入
            # for x in fe:
            #     print(x, file=doc)  #将打印内容写入文件中

            ob = np.load(obstacle_path, allow_pickle=True)
            la = np.load(label_path, allow_pickle=True)
            hetero_graphs = []

            for i in range(len(ti)):
                ntypes = ['line', 'location', 'obstacle']
                etypes = [('line', 'linetoloc', 'location'), ('location', 'loctoline', 'line'),
                          ('obstacle', 'obstoloc', 'location'), ('location', 'loctoobs', 'obstacle')]
                number_of_line = len(fe[i])
                number_of_obstacle = len(ob[i])

                line_to_loc1 = torch.tensor([[j] * (number_of_obstacle) for j in range(number_of_line)],dtype=torch.long).view(-1)
                line_to_loc2 = torch.tensor([j for j in range(number_of_line * number_of_obstacle)], dtype=torch.long)

                obs_to_loc1 = torch.tensor([[j] * (number_of_line) for j in range(number_of_obstacle)], dtype=torch.long).view(-1)
                o_l2 = []
                
                for j in range(number_of_obstacle):
                    for k in range(j, number_of_line * number_of_obstacle, number_of_obstacle):
                        o_l2.append(k)
                obs_to_loc2 = torch.tensor(o_l2, dtype=torch.long)

                obstacle_to_line = []
                for j in range(number_of_obstacle):
                    for k in range(number_of_line):
                        obstacle_to_line.append((j, k))

                line_position = []
                for j in range(number_of_line):
                    line_position.append(node_location[0] + node_location[j + 1])
                obstacle_feature = []
                obstacle_position = []

                for k in range(len(ob[i])):
                    oof = ob[i][k]
                    if oof[0] == -1:
                        obstacle_feature.append([-1, -1, -1])
                        obstacle_position.append([-1, -1])
                    else:
                        obstacle_feature.append(
                            # P, W * H, W / H
                            [oof[0], (oof[3] - oof[1]) * (oof[4] - oof[2]), (oof[3] - oof[1]) / (oof[4] - oof[2])])
                        obstacle_position.append([(oof[3] + oof[1]) / 2, -(oof[4] - oof[2]) / 2])
                e_f = []

                for j in range(number_of_obstacle):  #
                    for k in range(number_of_line):
                        line_x1 = line_position[k][0]
                        line_y1 = line_position[k][1]
                        line_x2 = line_position[k][2]
                        line_y2 = line_position[k][3]
                        point_x = obstacle_position[j][0]
                        point_y = obstacle_position[j][1]
                        if (point_x == -1 and point_y == -1) or (line_x2 == -999 and line_y2 == -999):
                            e_f.append([-1, -1, -1])
                        else:
                            dis1 = distance(line_x1, line_y1, point_x, point_y)
                            dis2 = distance(line_x2, line_y2, point_x, point_y)
                            dis3 = distance(line_x1, line_y1, line_x2, line_y2)
                            e_f.append([dis1, dis2, dis3])
                loc_features = torch.tensor(e_f, dtype=torch.float)

                hetero_graph = dgl.heterograph({
                    ('line', 'linetoloc', 'location'): (line_to_loc1, line_to_loc2),
                    ('location', 'loctoline', 'line'): (line_to_loc2, line_to_loc1),
                    ('obstacle', 'obstoloc', 'location'): (obs_to_loc1, obs_to_loc2),
                    ('location', 'loctoobs', 'obstacle'): (obs_to_loc2, obs_to_loc1)
                },
                    num_nodes_dict={
                        'line': number_of_line,
                        'location': number_of_line * number_of_obstacle,
                        'obstacle': number_of_obstacle
                    }
                )
                hetero_graph.nodes['line'].data['feat'] = torch.tensor(fe[i], dtype=torch.float)
                hetero_graph.nodes['obstacle'].data['feat'] = torch.tensor(obstacle_feature, dtype=torch.float)
                hetero_graph.nodes['location'].data['feat'] = loc_features
                hetero_graphs.append(hetero_graph)
            dgl.save_graphs(graph_path, hetero_graphs)
print('graph saved success')

### 取1秒内10个时刻的图制作为异构图

In [ ]:
from extension.data_process import construct_htg_covid, load_data
op_type = ['train', 'bin_test', 'ran_test']
time_window = 10
for op in op_type:
    if op == 'train':
        # 标签加载路径
        load_str = 'D:\\GCN\\graphnet\\data\\train_data\\'
        # 大图输出路径
        graph_folder = '.\\train_data\\'
        max_sc_no = 6
    elif op == 'bin_test':
        load_str = 'D:\\GCN\\graphnet\\data\\test_data2\\'
        graph_folder = '.\\bin_test_data\\'
        max_sc_no = 4
    elif op == 'ran_test':
        load_str = 'D:\\GCN\\graphnet\\data\\test_data2\\'
        graph_folder = '.\\ran_test_data\\'
        max_sc_no = 4
    for scene in ['indoor', 'outdoor']:
        for numid in range(max_sc_no):
            print('saving scene:', scene, 'trace: ', numid + 1)
            label_path = load_str + scene + '\\30-' + str(numid + 1) + '\\dataset\\label.npy'
            graph_path = graph_folder + scene + '_30-' + str(numid + 1) + '_graphs.dgl'
            label, feat, index = [], [], []
            la = np.load(label_path, allow_pickle=True)
            g_list, _ = dgl.load_graphs(graph_path)
            feat, label, index = load_data(g_list, la, time_window)

            dgl.save_graphs(graph_folder + scene + '_30-' + str(numid + 1) + '_graphs_feature.dgl', feat)
            np.save(graph_folder + scene + '_30-' + str(numid + 1) + '_graphs_label.npy', label)
            np.save(graph_folder + scene + '_30-' + str(numid + 1) + '_index.npy', index)